In [1]:
import gc_utils
import gizmo_analysis as gizmo
import h5py
import halo_analysis as halo
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.interpolate as interp
import scipy.signal as signal
import utilities as ut


In [2]:
sim = "m12i"
sim_dir = "/Users/z5114326/Documents/simulations/"
fire_dir = sim_dir + sim + "/" + sim + "_res7100"

snap_dir = fire_dir + "/snapshot_times.txt"
snap_data = pd.read_table(snap_dir, comment="#", header=None, sep=r"\s+")
snap_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]

snap_pub_dir = sim_dir + "/snapshot_times_public.txt"
snap_pub_data = pd.read_table(snap_pub_dir, comment="#", header=None, sep=r"\s+")
snap_pub_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]

In [3]:
def get_snap_groups(it_min: int, it_max: int, snapshot: int, proc_data):
    snap_id = gc_utils.snapshot_name(snapshot)

    snap_groups = set()

    for it in range(it_min, it_max + 1):
        it_id = gc_utils.iteration_name(it)

        snap_dat = proc_data[it_id]["snapshots"][snap_id]

        grp_lst = np.abs(snap_dat["group_id"][()])
        snap_groups.update(grp_lst)

    snap_groups = np.array(sorted(snap_groups))

    return snap_groups

In [4]:
def get_acc_snap(halo_tid, main_halo_tid, halt):
    tid_main_lst = gc_utils.main_prog_halt(halt, main_halo_tid)
    desc_lst = gc_utils.get_descendants_halt(halo_tid, halt)

    idx_lst = np.array([1 if halt["tid"][idx] in tid_main_lst else 0 for idx in desc_lst])
    idx_acc = np.where(idx_lst == 1)[0][0]

    snap_acc = halt["snapshot"][desc_lst[idx_acc]]

    return snap_acc

In [5]:
def get_galaxy_at_snap(grp_val, snap_final, main_halo_tid, halt):
    if grp_val != 0:
        idx = np.where(halt["tid"] == grp_val)[0][0]
        snap = halt["snapshot"][idx]

        acc_snap = get_acc_snap(grp_val, main_halo_tid, halt)

        if snap < acc_snap:
            while snap > snap_final:
                idx = halt["progenitor.main.index"][idx]
                snap = halt["snapshot"][idx]

        else:
            idx = 0
            snap = halt["snapshot"][idx]

            while snap > snap_final:
                idx = halt["progenitor.main.index"][idx]
                snap = halt["snapshot"][idx]

    else:
        idx = 0
        snap = halt["snapshot"][idx]

        while snap > snap_final:
            idx = halt["progenitor.main.index"][idx]
            snap = halt["snapshot"][idx]

    galaxy_tid = halt["tid"][idx]
    galaxy_idx = idx

    # print(snap, host_tid)

    return galaxy_tid, galaxy_idx

In [6]:
def get_group_particle_lst(it, snapshot, group_id, proc_data):
    it_id = gc_utils.iteration_name(it)
    snap_id = gc_utils.snapshot_name(snapshot)
    snap_data = proc_data[it_id]["snapshots"][snap_id]

    # we use absolute to account for -ve group values (destroyed before accretion)
    grp_mask = np.abs(snap_data["group_id"][()]) == group_id

    gc_ids = snap_data["gc_id"][()][grp_mask]

    ptypes_bin = snap_data["ptype"][()][grp_mask]
    ptypes = np.array([ptype.decode("utf-8") for ptype in ptypes_bin])

    return gc_ids, ptypes

In [7]:
# inputs ################################
sim = "m12i"
sim_dir = "/Users/z5114326/Documents/simulations/"

main_halo_tid = 25236877

halt = gc_utils.get_halo_tree(sim, sim_dir, assign_hosts_rotation=True)

Retrieving Halo Tree.....................: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.32s/it]


In [8]:
it_min = 0
it_max = 0

snapshot = 600

In [9]:
# function ##############################

fire_dir = sim_dir + sim + "/" + sim + "_res7100"

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file


In [10]:
snap_groups = get_snap_groups(it_min, it_max, snapshot, proc_data)

In [11]:
snap_groups = [snap_groups[0]]
group_id = snap_groups[0]

# for grp in snap_groups:
galaxy_tid, galaxy_tree_idx = get_galaxy_at_snap(group_id, snapshot, main_halo_tid, halt)

In [12]:
part = gc_utils.open_snapshot(snapshot, fire_dir, species=["dark", "star"], assign_hosts_rotation=True)

Retrieving Snapshot 600..................:   0%|                                                                                | 0/1 [00:06<?, ?it/s]


KeyboardInterrupt: 

# Lets get the group information:

In [ ]:
# get rotation tensor
rotate_tensor = part.host["rotation"][0]
unrotate_tensor = rotate_tensor.transpose()

In [ ]:
halt_snap_mask = halt["snapshot"] == snapshot
host_tree_idx = np.unique(halt["host.index"][halt_snap_mask])[0]

In [ ]:
# position of galaxy in reference to host

galaxy_pos_host_unrotated = ut.coordinate.get_distances(
    halt["position"][galaxy_tree_idx],
    halt["position"][host_tree_idx],
    part.info["box.length"],
    part.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

galaxy_pos_host_rotated = ut.coordinate.get_coordinates_rotated(galaxy_pos_host_unrotated, rotate_tensor)

In [ ]:
rotate_tensor

array([[-0.14789672, -0.9403196 ,  0.30647293],
       [-0.8559442 ,  0.27694073,  0.4366501 ],
       [-0.4954655 , -0.19774461, -0.8458197 ]], dtype=float32)

In [ ]:
galaxy_pos_host_rotated

array([0., 0., 0.], dtype=float32)

# Lets get the particle information

In [ ]:
it = it_min  # temporary

gc_ids, ptypes = get_group_particle_lst(it, snapshot, group_id, proc_data)

In [ ]:
part_dict = {}
part_dict["gc_id"] = gc_ids
part_dict["ptype"] = ptypes
part_dict["part_idx"] = []
part_dict["pos"] = []
# part_dict["vel"] = []

for gc_id, ptype in zip(gc_ids, ptypes):
    gc_part_idx = np.where(part[ptype]["id"] == gc_id)[0][0]

    part_pos_host_unrotated = ut.coordinate.get_distances(
        part[ptype]["position"][gc_part_idx],
        halt["position"][host_tree_idx],
        part.info["box.length"],
        part.snapshot["scalefactor"],
        total_distance=False,
    )  # [kpc physical]

    part_pos_host_rotated = ut.coordinate.get_coordinates_rotated(part_pos_host_unrotated, rotate_tensor)

    part_dict["part_idx"].append(gc_part_idx)
    part_dict["pos"].append(part_pos_host_rotated)

In [ ]:
part_dict["gc_id"][0], part_dict["pos"][0], part_dict["ptype"][0]

(57057454, array([-1.35912218,  1.49856192, -0.95284276]), 'star')

In [ ]:
proc_data["it000"]["snapshots"]["snap077"]["x"][0]

55.69042469678172

In [ ]:
part[part_dict["ptype"][0]].prop("host.distance.principal", part_dict["part_idx"][0])

array([-1.33877489,  1.47705912, -0.96118916])

TEST v2

In [ ]:
dist = ut.particle.get_distances_wrt_center(
    part,
    species=["dark", "star"],
    center_position=halt["position"][host_tree_idx],
    rotation=True,
    total_distance=True,
)

sp = "star"
rmax_ctr = 10

ctr_indices = np.where(dist[sp] < rmax_ctr)[0]

m = part[sp]["mass"][ctr_indices]
pos = part[sp]["position"][ctr_indices]
vel = part[sp]["velocity"][ctr_indices]
new_ctr = np.multiply(m, pos.T).sum(axis=1) / np.sum(m)
new_vctr = np.multiply(m, vel.T).sum(axis=1) / np.sum(m)

new_rot = ut.particle.get_principal_axes(
    part,
    species_name="star",
    distance_max=rmax_ctr,
    mass_percent=90,
    age_percent=25,
    age_limits=None,
    temperature_limits=None,
    center_positions=new_ctr,
    center_velocities=new_vctr,
    host_index=None,
    part_indicess=None,
    return_single_array=True,
    verbose=None,
)

dist_vect = ut.particle.get_distances_wrt_center(
    part,
    species=["dark", "star"],
    center_position=new_ctr,
    rotation=new_rot["rotation"],
)

vel_vect = ut.particle.get_velocities_wrt_center(
    part,
    species=["dark", "star"],
    center_position=new_ctr,
    center_velocity=new_vctr,
    rotation=new_rot["rotation"],
)

In [ ]:
dist_vect

{'dark': array([[ -185.49260658, -1086.69984043, -1320.53833179],
        [ -186.36917064, -1087.70792362, -1320.49359287],
        [ -186.8556588 , -1087.22228407, -1320.42098016],
        ...,
        [-1091.1739506 ,  1285.26723112,   425.24161947],
        [-1107.93921914,  1297.96058485,   429.11101053],
        [-1124.38864848,  1318.02724933,   431.04990922]]),
 'star': array([[ 1.89555929e+03, -2.98121533e+04,  1.94063986e+02],
        [ 1.43246327e+01, -2.41978971e+03, -3.88508117e+03],
        [-1.03923575e+03, -2.78212628e+03, -4.51937600e+03],
        ...,
        [ 2.20969959e+03, -2.78697665e+01,  1.12855762e+03],
        [ 2.28511561e+03, -2.59181549e+01,  1.36052784e+03],
        [ 2.32731419e+03, -2.89441485e+01,  1.32654901e+03]])}

In [ ]:
part["dark"].prop("host.distance.principal")[0]

array([ -157.00519938, -1091.15235097, -1320.5879644 ])